In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
df=pd.read_csv("../input/unsupervised-learning-on-country-data/Country-data.csv")

In [ ]:
print(df.isna().sum())

In [ ]:
df.head()

In [ ]:
#Label encoding gender column
from sklearn.preprocessing import LabelEncoder
lec=LabelEncoder()
new_df['country'] = lec.fit_transform(df['country'])

In [ ]:
df.head()

In [ ]:
new_df=df

In [ ]:
col=['child_mort','exports','health','imports','income','inflation','life_expec','total_fer','gdpp']
hist=df[col].hist(figsize=(12,6))
plt.show()

In [ ]:
for i,c in enumerate(col):
    print("Skew of {} is {}".format(c,df[c].skew()))

In [ ]:
from sklearn.preprocessing import FunctionTransformer
log_tf = FunctionTransformer(np.log1p)
a=log_tf.transform(df.income)
plt.hist(a, bins = 10)

In [ ]:
from scipy import stats
life_boxcox=stats.boxcox(df['life_expec'])[0]
plt.hist(life_boxcox, bins = 10)

In [ ]:
new_df['life_expec']=life_boxcox

In [ ]:
col1=['child_mort','exports','health','imports','income','total_fer','gdpp']
for i,c in enumerate(col1):
    a=log_tf.transform(df[c])
    new_df[c]=a
    

In [ ]:
sqrt_tf = FunctionTransformer(np.sqrt)
a=log_tf.transform(df.inflation)
df.inflation=a

In [ ]:
for i,c in enumerate(col):
    print("Skew of {} is {}".format(c,new_df[c].skew()))

In [ ]:
hist=new_df[col].hist(figsize=(12,6))
plt.show()

In [ ]:
new_df=new_df.drop('country',axis=1)

In [ ]:
print(new_df.isna().sum())

In [ ]:
from sklearn.impute import SimpleImputer
imputer=SimpleImputer(missing_values=np.nan,strategy='mean')
imputer = imputer.fit(new_df[['inflation']])
new_df['inflation'] = imputer.transform(new_df[['inflation']]).ravel()

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
inertia = []
score=[]
K1 = range(1,10)
for k in K1:
    kmean = KMeans(n_clusters=k)
    kmean.fit(new_df)
    inertia.append(kmean.inertia_)
K2 = range(2,10)
for k in K2:
    kmean = KMeans(n_clusters=k)
    kmean.fit(new_df)
    preds = kmean.predict(new_df)
    x=silhouette_score(new_df, kmean.labels_)
    score.append(x)

In [ ]:

plt.figure(figsize=(20,10))
plt.subplot(121)
plt.plot(K1, inertia,'bx-',c='red')
plt.xlabel('clusters')
plt.ylabel('Inertia')
plt.title('The Elbow Method showing the optimal k')
plt.subplot(122)
plt.plot(K2, score,'bx-', c='blue')
plt.xlabel('clusters')
plt.ylabel('Silhoutte Score')
plt.title('The Silhoutte Score showing the optimal k')
plt.show()

In [ ]:
km = KMeans(n_clusters=4)
clusters = km.fit_predict(new_df.iloc[:,1:])
new_df["k_label"] = clusters

In [ ]:
new_df['k_label'].value_counts()

In [ ]:
fig = plt.figure(figsize=(20,10))
ax = fig.add_subplot(111, projection='3d')
for i in range(5): 
  ax.scatter( new_df["income"][new_df.k_label == i],new_df["health"][new_df.k_label == i],new_df.child_mort[new_df.k_label == i], s=30)
# ax.view_init(30, 185)
plt.ylabel("income")
plt.xlabel("health")
ax.set_zlabel('child_mort')
plt.legend(np.unique(df.k_label))
plt.show()